# Geospatial Visualization

## Geojson Data
  
Import Libraries

In [2]:
# Import Libraries
import pandas as pd
import numpy as np
import geopandas as gpd
import folium as flm
import calendar
#  For showing all columns in Pandas
pd.set_option('display.max_columns', None)

# this ignores the depreciation warnings etc
import warnings
warnings.filterwarnings("ignore")

### Create a Dataframe contianing geometry of the Police Station Areas
  
Read in the data and create a DataFrame.

In [3]:
# Read the geoJSON file using geopandas
geo_prov = gpd.read_file(r'../../../data/geodata/sa_provinces.geojson')
geo_prov = geo_prov[["ADM1_ID", "ADM1_EN", "geometry"]] # only select 'COMPNT_NM' (Police Stations) and 'geometry' columns

In [3]:
geo_prov

,ADM1_ID,ADM1_EN,geometry
0,EC,Eastern Cape,"POLYGON ((30.19386 -31.08126, 30.19386 -31.081..."
1,FS,Free State,"POLYGON ((28.24428 -26.88478, 28.24348 -26.884..."
2,GT,Gauteng,"POLYGON ((28.24428 -26.88478, 28.24696 -26.884..."
3,KZN,KwaZulu-Natal,"POLYGON ((30.19386 -31.08126, 30.19382 -31.081..."
4,LIM,Limpopo,"POLYGON ((31.88383 -23.98459, 31.88092 -23.967..."
5,MP,Mpumalanga,"POLYGON ((31.88383 -23.98459, 31.85268 -23.986..."
6,NW,North West,"POLYGON ((28.29816 -25.31037, 28.29835 -25.294..."
7,NC,Nothern Cape,"POLYGON ((22.63217 -26.12128, 22.63196 -26.121..."
8,WC,Western Cape,"MULTIPOLYGON (((19.41807 -34.68668, 19.41806 -..."


Fix spelling mistake - 'Nothern Cape' to 'Northern Cape'.

In [4]:
geo_prov['ADM1_EN'] = geo_prov['ADM1_EN'].replace('Nothern Cape','Northern Cape')
geo_prov

,ADM1_ID,ADM1_EN,geometry
0,EC,Eastern Cape,"POLYGON ((30.19386 -31.08126, 30.19386 -31.081..."
1,FS,Free State,"POLYGON ((28.24428 -26.88478, 28.24348 -26.884..."
2,GT,Gauteng,"POLYGON ((28.24428 -26.88478, 28.24696 -26.884..."
3,KZN,KwaZulu-Natal,"POLYGON ((30.19386 -31.08126, 30.19382 -31.081..."
4,LIM,Limpopo,"POLYGON ((31.88383 -23.98459, 31.88092 -23.967..."
5,MP,Mpumalanga,"POLYGON ((31.88383 -23.98459, 31.85268 -23.986..."
6,NW,North West,"POLYGON ((28.29816 -25.31037, 28.29835 -25.294..."
7,NC,Northern Cape,"POLYGON ((22.63217 -26.12128, 22.63196 -26.121..."
8,WC,Western Cape,"MULTIPOLYGON (((19.41807 -34.68668, 19.41806 -..."


In [5]:
geo_prov.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   ADM1_ID   9 non-null      object  
 1   ADM1_EN   9 non-null      object  
 2   geometry  9 non-null      geometry
dtypes: geometry(1), object(2)
memory usage: 344.0+ bytes


### Create a Dataframe containing the Police Station Crime Data
  
Read in the data and create a DataFrame.

In [6]:
df = pd.read_parquet('../../../data/crime_data_2016_21.parquet')

In [7]:
df

,station,province,district,crime_category,date,number_of_crimes,latitude,longitude
0,East London,Eastern Cape,East London Cc,17 Community Reported Serious Crime,Jan-16,470,-33.02058,27.90288
1,East London,Eastern Cape,East London Cc,17 Community Reported Serious Crime,Feb-16,411,-33.02058,27.90288
2,East London,Eastern Cape,East London Cc,17 Community Reported Serious Crime,Mar-16,477,-33.02058,27.90288
3,East London,Eastern Cape,East London Cc,17 Community Reported Serious Crime,Jan-17,476,-33.02058,27.90288
4,East London,Eastern Cape,East London Cc,17 Community Reported Serious Crime,Feb-17,427,-33.02058,27.90288
...,...,...,...,...,...,...,...,...
3665371,Protea Glen,Gauteng,Soweto West Cc,Truck hijacking,Nov-20,0,-26.27697,27.83896
3665372,Protea Glen,Gauteng,Soweto West Cc,Truck hijacking,Dec-20,0,-26.27697,27.83896
3665373,Protea Glen,Gauteng,Soweto West Cc,Truck hijacking,Oct-21,0,-26.27697,27.83896
3665374,Protea Glen,Gauteng,Soweto West Cc,Truck hijacking,Nov-21,0,-26.27697,27.83896


Lets get the geometric district names

In [8]:
df2 = pd.read_csv('district_municipals.csv')
df2  = pd.DataFrame(df2[['district', 'main_district']])
df2

,district,main_district
0,Mount Ayliff Cc,Alfred Nzo
1,Butterworth Cc,Amathole
2,Alice Cc,Amathole
3,Rustenburg Cc,Bojanala
4,Brits Cc,Bojanala
...,...,...
117,Zf Mgcawu Cc,Z F Mgcawu
118,King Cetshwayo Cc,Uthungulu
119,Ilembe Cc,iLembe
120,Amajuba Cc,Zululand


Lets merge with the DataFrame.

In [9]:
df = df.merge(df2, on='district', how='left')
df

,station,province,district,crime_category,date,number_of_crimes,latitude,longitude,main_district
0,East London,Eastern Cape,East London Cc,17 Community Reported Serious Crime,Jan-16,470,-33.02058,27.90288,Buffalo City
1,East London,Eastern Cape,East London Cc,17 Community Reported Serious Crime,Feb-16,411,-33.02058,27.90288,Buffalo City
2,East London,Eastern Cape,East London Cc,17 Community Reported Serious Crime,Mar-16,477,-33.02058,27.90288,Buffalo City
3,East London,Eastern Cape,East London Cc,17 Community Reported Serious Crime,Jan-17,476,-33.02058,27.90288,Buffalo City
4,East London,Eastern Cape,East London Cc,17 Community Reported Serious Crime,Feb-17,427,-33.02058,27.90288,Buffalo City
...,...,...,...,...,...,...,...,...,...
3665371,Protea Glen,Gauteng,Soweto West Cc,Truck hijacking,Nov-20,0,-26.27697,27.83896,City of Johannesburg
3665372,Protea Glen,Gauteng,Soweto West Cc,Truck hijacking,Dec-20,0,-26.27697,27.83896,City of Johannesburg
3665373,Protea Glen,Gauteng,Soweto West Cc,Truck hijacking,Oct-21,0,-26.27697,27.83896,City of Johannesburg
3665374,Protea Glen,Gauteng,Soweto West Cc,Truck hijacking,Nov-21,0,-26.27697,27.83896,City of Johannesburg


Lets do a bit of housekeeping to tidy things up.

In [10]:
df.columns

Index(['station', 'province', 'district', 'crime_category', 'date',
       'number_of_crimes', 'latitude', 'longitude', 'main_district'],
      dtype='object')

In [11]:
df = df[
    ['station', 'district', 'main_district', 'province', 'date',
     'crime_category', 'number_of_crimes', 'latitude', 'longitude',]]
df.head(1)

,station,district,main_district,province,date,crime_category,number_of_crimes,latitude,longitude
0,East London,East London Cc,Buffalo City,Eastern Cape,Jan-16,17 Community Reported Serious Crime,470,-33.02058,27.90288


In [12]:
df.rename(columns = {'district':'municipality', 'main_district': 'district'}, inplace = True)
df.head(1)

,station,municipality,district,province,date,crime_category,number_of_crimes,latitude,longitude
0,East London,East London Cc,Buffalo City,Eastern Cape,Jan-16,17 Community Reported Serious Crime,470,-33.02058,27.90288


Change the format of 'Kwazulu/Natal' to match 'Kwazulu-Natal'.

In [14]:
df['province'] = df['province'].replace('Kwazulu/Natal','KwaZulu-Natal')
df.sample(10)

,station,municipality,district,province,date,crime_category,number_of_crimes,latitude,longitude
955048,Goedemoed,Smithfield Cc,Xhariep,Free State,May-17,Shoplifting,0,-30.55924,26.40335
2073270,Makhado,Makhado Cc,Vhembe,Limpopo,Jul-20,Robbery of cash in transit,0,-23.03789,29.90773
2016320,Rietvlei,Umgungundlovo N Cc,Ehlanzeni,KwaZulu-Natal,Sep-20,Sexual offences detected as a result of police...,0,-29.17992,30.34741
2069165,Malamulele,Giyani Cc,Mopani,Limpopo,Jun-19,Murder,1,-23.00452,30.68665
1913073,Gowan Lea,Harry Gwala Cc,Sisonke,KwaZulu-Natal,Jan-21,Drug-Related Crime,0,-30.34742,29.70266
624635,Doringkloof,Alice Cc,Amathole,Eastern Cape,Mar-21,Public violence,0,-32.45969,26.51463
81705,Duncan Village,East London Cc,Buffalo City,Eastern Cape,Oct-17,Bank Robbery,0,-33.00505,27.88872
1922824,Nhlanhleni,Umzinyathi Cc,Umzinyathi,KwaZulu-Natal,Feb-19,Sexual Assault,0,-28.55147,30.42224
604340,Khubusidrift,King William'S T Cc,O.R.Tambo,Eastern Cape,Dec-18,All theft not mentioned elsewhere,0,-32.56809,27.61970
691211,Selosesha,Selosesha Cc,Mangaung,Free State,Mar-19,Robbery at residential premises,1,-29.19424,26.81939


Lets check the shape of the dataframe and the length of 'province'.

In [15]:
len(df['province'].unique())

9

Quick check of the shape confirms all is good.

In [16]:
geo_prov.shape

(9, 3)

Lets check the column names.

In [17]:
geo_prov.columns

Index(['ADM1_ID', 'ADM1_EN', 'geometry'], dtype='object')

We will renmane the column 'ADM1_EN' to 'province', and 'geometry' to 'geometry_prov'

In [18]:
geo_prov.rename(columns = {'ADM1_EN':'province', 'geometry': 'geometry_prov'}, inplace = True)
geo_prov

,ADM1_ID,province,geometry_prov
0,EC,Eastern Cape,"POLYGON ((30.19386 -31.08126, 30.19386 -31.081..."
1,FS,Free State,"POLYGON ((28.24428 -26.88478, 28.24348 -26.884..."
2,GT,Gauteng,"POLYGON ((28.24428 -26.88478, 28.24696 -26.884..."
3,KZN,KwaZulu-Natal,"POLYGON ((30.19386 -31.08126, 30.19382 -31.081..."
4,LIM,Limpopo,"POLYGON ((31.88383 -23.98459, 31.88092 -23.967..."
5,MP,Mpumalanga,"POLYGON ((31.88383 -23.98459, 31.85268 -23.986..."
6,NW,North West,"POLYGON ((28.29816 -25.31037, 28.29835 -25.294..."
7,NC,Northern Cape,"POLYGON ((22.63217 -26.12128, 22.63196 -26.121..."
8,WC,Western Cape,"MULTIPOLYGON (((19.41807 -34.68668, 19.41806 -..."


The 'geo_prov' data matches the 'df' 'DataFrame'

Now we can check the data types.

In [19]:
geo_prov.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   ADM1_ID        9 non-null      object  
 1   province       9 non-null      object  
 2   geometry_prov  9 non-null      geometry
dtypes: geometry(1), object(2)
memory usage: 344.0+ bytes


In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3665376 entries, 0 to 3665375
Data columns (total 9 columns):
 #   Column            Dtype  
---  ------            -----  
 0   station           object 
 1   municipality      object 
 2   district          object 
 3   province          object 
 4   date              object 
 5   crime_category    object 
 6   number_of_crimes  int32  
 7   latitude          float64
 8   longitude         float64
dtypes: float64(2), int32(1), object(6)
memory usage: 265.7+ MB


## Merge the DataFrames

Lets create a new DataFrame of the merged DataFrames.

In [21]:
geospatial_main = geo_prov.merge(df, on=['province'], how='left')

In [22]:
geospatial_main

,ADM1_ID,province,geometry_prov,station,municipality,district,date,crime_category,number_of_crimes,latitude,longitude
0,EC,Eastern Cape,"POLYGON ((30.19386 -31.08126, 30.19386 -31.081...",East London,East London Cc,Buffalo City,Jan-16,17 Community Reported Serious Crime,470,-33.02058,27.90288
1,EC,Eastern Cape,"POLYGON ((30.19386 -31.08126, 30.19386 -31.081...",East London,East London Cc,Buffalo City,Feb-16,17 Community Reported Serious Crime,411,-33.02058,27.90288
2,EC,Eastern Cape,"POLYGON ((30.19386 -31.08126, 30.19386 -31.081...",East London,East London Cc,Buffalo City,Mar-16,17 Community Reported Serious Crime,477,-33.02058,27.90288
3,EC,Eastern Cape,"POLYGON ((30.19386 -31.08126, 30.19386 -31.081...",East London,East London Cc,Buffalo City,Jan-17,17 Community Reported Serious Crime,476,-33.02058,27.90288
4,EC,Eastern Cape,"POLYGON ((30.19386 -31.08126, 30.19386 -31.081...",East London,East London Cc,Buffalo City,Feb-17,17 Community Reported Serious Crime,427,-33.02058,27.90288
...,...,...,...,...,...,...,...,...,...,...,...
3665371,WC,Western Cape,"MULTIPOLYGON (((19.41807 -34.68668, 19.41806 -...",Int Airport C Town,East Metropol,City of Cape Town,Nov-20,Truck hijacking,0,-33.97146,18.59990
3665372,WC,Western Cape,"MULTIPOLYGON (((19.41807 -34.68668, 19.41806 -...",Int Airport C Town,East Metropol,City of Cape Town,Dec-20,Truck hijacking,0,-33.97146,18.59990
3665373,WC,Western Cape,"MULTIPOLYGON (((19.41807 -34.68668, 19.41806 -...",Int Airport C Town,East Metropol,City of Cape Town,Oct-21,Truck hijacking,0,-33.97146,18.59990
3665374,WC,Western Cape,"MULTIPOLYGON (((19.41807 -34.68668, 19.41806 -...",Int Airport C Town,East Metropol,City of Cape Town,Nov-21,Truck hijacking,0,-33.97146,18.59990


Convert to a GeoPandas DataFrame.

In [23]:
geospatial_main = gpd.GeoDataFrame(geospatial_main, geometry='geometry_prov')

In [24]:
type(geospatial_main)

geopandas.geodataframe.GeoDataFrame

Lets create a matching 'prov_id' from 'ADM1_ID'.

In [25]:
geospatial_main['prov_id'] = geospatial_main.loc[:, 'ADM1_ID']

In [26]:
geospatial_main.sample(2)

,ADM1_ID,province,geometry_prov,station,municipality,district,date,crime_category,number_of_crimes,latitude,longitude,prov_id
12763,EC,Eastern Cape,"POLYGON ((30.19386 -31.08126, 30.19386 -31.081...",Humewood,Mount Road Cc,Nelson Mandela Bay,Feb-16,Attempted murder,0,-33.97403,25.62617,EC
3201864,WC,Western Cape,"MULTIPOLYGON (((19.41807 -34.68668, 19.41806 -...",Mfuleni,Blue Downs Cc,City of Cape Town,Jul-18,Robbery of cash in transit,0,-33.98106,18.68742,WC


Lets change the date to 'datetime'.

In [27]:
geospatial_main['date'] = pd.to_datetime(geospatial_main['date'], format='%b-%y')

Quick check.

In [28]:
geospatial_main.head(1)

,ADM1_ID,province,geometry_prov,station,municipality,district,date,crime_category,number_of_crimes,latitude,longitude,prov_id
0,EC,Eastern Cape,"POLYGON ((30.19386 -31.08126, 30.19386 -31.081...",East London,East London Cc,Buffalo City,2016-01-01,17 Community Reported Serious Crime,470,-33.02058,27.90288,EC


### Create Month and Year columns

In [29]:
geospatial_main['month'] = geospatial_main['date'].apply(lambda x: x.month)
geospatial_main['month'] = geospatial_main['month'].apply(lambda x: calendar.month_abbr[x])
geospatial_main['year'] = geospatial_main['date'].apply(lambda x: x.year)
geospatial_main.head(1)

,ADM1_ID,province,geometry_prov,station,municipality,district,date,crime_category,number_of_crimes,latitude,longitude,prov_id,month,year
0,EC,Eastern Cape,"POLYGON ((30.19386 -31.08126, 30.19386 -31.081...",East London,East London Cc,Buffalo City,2016-01-01,17 Community Reported Serious Crime,470,-33.02058,27.90288,EC,Jan,2016


Reorder columns

In [30]:

geospatial_main = geospatial_main[
    ['station', 'district', 'province', 'prov_id',
     'crime_category', 'date', 'month', 'year',
     'number_of_crimes', 'latitude', 'longitude',
     'ADM1_ID', 'geometry_prov']]
geospatial_main.head(1)

,station,district,province,prov_id,crime_category,date,month,year,number_of_crimes,latitude,longitude,ADM1_ID,geometry_prov
0,East London,Buffalo City,Eastern Cape,EC,17 Community Reported Serious Crime,2016-01-01,Jan,2016,470,-33.02058,27.90288,EC,"POLYGON ((30.19386 -31.08126, 30.19386 -31.081..."


In [31]:
geospatial_main['year'].info()

<class 'pandas.core.series.Series'>
Int64Index: 3665376 entries, 0 to 3665375
Series name: year
Non-Null Count    Dtype
--------------    -----
3665376 non-null  int64
dtypes: int64(1)
memory usage: 55.9 MB


Lets change the year to a string.

In [32]:
geospatial_main['year'] = geospatial_main['year'].map(str)

In [33]:
geospatial_main['year'].info()

<class 'pandas.core.series.Series'>
Int64Index: 3665376 entries, 0 to 3665375
Series name: year
Non-Null Count    Dtype 
--------------    ----- 
3665376 non-null  object
dtypes: object(1)
memory usage: 55.9+ MB


## Mapping
Lets create a dataframe for the Folium map.

### Yearly Data

In [34]:
map1 = geospatial_main.groupby(['prov_id', 'province', 'year', 'ADM1_ID'], as_index=False).agg({'number_of_crimes': 'sum'})
map1.head()

,prov_id,province,year,ADM1_ID,number_of_crimes
0,EC,Eastern Cape,2016,EC,587452
1,EC,Eastern Cape,2017,EC,570766
2,EC,Eastern Cape,2018,EC,578181
3,EC,Eastern Cape,2019,EC,579301
4,EC,Eastern Cape,2020,EC,500245


In [35]:
map1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54 entries, 0 to 53
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   prov_id           54 non-null     object
 1   province          54 non-null     object
 2   year              54 non-null     object
 3   ADM1_ID           54 non-null     object
 4   number_of_crimes  54 non-null     int32 
dtypes: int32(1), object(4)
memory usage: 2.0+ KB


In [36]:
geo_prov

,ADM1_ID,province,geometry_prov
0,EC,Eastern Cape,"POLYGON ((30.19386 -31.08126, 30.19386 -31.081..."
1,FS,Free State,"POLYGON ((28.24428 -26.88478, 28.24348 -26.884..."
2,GT,Gauteng,"POLYGON ((28.24428 -26.88478, 28.24696 -26.884..."
3,KZN,KwaZulu-Natal,"POLYGON ((30.19386 -31.08126, 30.19382 -31.081..."
4,LIM,Limpopo,"POLYGON ((31.88383 -23.98459, 31.88092 -23.967..."
5,MP,Mpumalanga,"POLYGON ((31.88383 -23.98459, 31.85268 -23.986..."
6,NW,North West,"POLYGON ((28.29816 -25.31037, 28.29835 -25.294..."
7,NC,Northern Cape,"POLYGON ((22.63217 -26.12128, 22.63196 -26.121..."
8,WC,Western Cape,"MULTIPOLYGON (((19.41807 -34.68668, 19.41806 -..."


Merge Geometry into DataFrame.

In [37]:
mapped_prov = pd.merge(map1, geo_prov, on=['ADM1_ID'], how='left')
mapped_prov = mapped_prov[['prov_id', 'province_x', 'year', 'ADM1_ID', 'number_of_crimes', 'geometry_prov']]
mapped_prov.rename(columns = {'province_x':'province'}, inplace = True)
mapped_prov

,prov_id,province,year,ADM1_ID,number_of_crimes,geometry_prov
0,EC,Eastern Cape,2016,EC,587452,"POLYGON ((30.19386 -31.08126, 30.19386 -31.081..."
1,EC,Eastern Cape,2017,EC,570766,"POLYGON ((30.19386 -31.08126, 30.19386 -31.081..."
2,EC,Eastern Cape,2018,EC,578181,"POLYGON ((30.19386 -31.08126, 30.19386 -31.081..."
3,EC,Eastern Cape,2019,EC,579301,"POLYGON ((30.19386 -31.08126, 30.19386 -31.081..."
4,EC,Eastern Cape,2020,EC,500245,"POLYGON ((30.19386 -31.08126, 30.19386 -31.081..."
5,EC,Eastern Cape,2021,EC,516424,"POLYGON ((30.19386 -31.08126, 30.19386 -31.081..."
6,FS,Free State,2016,FS,340704,"POLYGON ((28.24428 -26.88478, 28.24348 -26.884..."
7,FS,Free State,2017,FS,320926,"POLYGON ((28.24428 -26.88478, 28.24348 -26.884..."
8,FS,Free State,2018,FS,318520,"POLYGON ((28.24428 -26.88478, 28.24348 -26.884..."
9,FS,Free State,2019,FS,305610,"POLYGON ((28.24428 -26.88478, 28.24348 -26.884..."


Convert to a GeoPandas DatFrame.

In [38]:
mapped_prov = gpd.GeoDataFrame(mapped_prov, geometry='geometry_prov')
type(mapped_prov)

geopandas.geodataframe.GeoDataFrame

In [ ]:
sa_map = flm.Map(location=[-28.343, 25.862], zoom_start=6, scrollWheelZoom=False, overlay=False, tiles=None)

flm.TileLayer('openstreetmap',name="Light Map",control=False).add_to(sa_map)

ft_2016 = mapped_prov[mapped_prov['year'] == '2016']
ft_2017 = mapped_prov[mapped_prov['year'] == '2017']
ft_2018 = mapped_prov[mapped_prov['year'] == '2018']
ft_2019 = mapped_prov[mapped_prov['year'] == '2019']
ft_2020 = mapped_prov[mapped_prov['year'] == '2020']
ft_2021 = mapped_prov[mapped_prov['year'] == '2021']

fg0 = flm.FeatureGroup(name='ft_2016',overlay=False).add_to(sa_map)
fg1 = flm.FeatureGroup(name='ft_2017',overlay=False).add_to(sa_map)
fg2 = flm.FeatureGroup(name='ft_2018',overlay=False).add_to(sa_map)
fg3 = flm.FeatureGroup(name='ft_2019',overlay=False).add_to(sa_map)
fg4 = flm.FeatureGroup(name='ft_2020',overlay=False).add_to(sa_map)
fg5 = flm.FeatureGroup(name='ft_2021',overlay=False).add_to(sa_map)

# fg1 = flm.FeatureGroup(name='Crimes Per Year', overlay=False).add_to(sa_map)

fs = [fg0, fg1, fg2, fg3, fg4, fg5]
year_data = [ft_2016, ft_2017, ft_2018, ft_2019, ft_2020, ft_2021]


custom_scale = (mapped_prov['number_of_crimes'].quantile((0,0.2,0.4,0.6,0.7,0.8,0.9,1))).tolist()

for i in range(len(year_data)):
    crimes_per_year = flm.Choropleth(
                geo_data=r'../../../data/geodata/sa_provinces.geojson',
                data=year_data[i],
                columns=['prov_id', 'number_of_crimes'],
                key_on='feature.properties.ADM1_ID',
                threshold_scale=custom_scale,
                fill_color='YlGnBu',
                nan_fill_color="blue",
                fill_opacity=0.5,
                line_opacity=0.2,
                legend_name='Number of Crimes ',
                highlight=True,
                line_color='black').geojson.add_to(fs[i])

    # Add customized tooltips to the map
    flm.features.GeoJson(
                        data = year_data[i],
                        name='Crimes Per Year',
                        smooth_factor=2,
                        style_function=lambda x: {'color':'black','fillColor':'transparent','weight':0.5},
                        tooltip=flm.features.GeoJsonTooltip(
                            fields=['province',
                                    'year',
                                    'number_of_crimes',
                                ],
                            aliases=["Province:",
                                    "Year",
                                    "Number of Crimes:",
                                    ],
                            localize=True,
                            sticky=False,
                            labels=True,
                            style="""
                                background-color: #F0EFEF;
                                border: 2px solid black;
                                border-radius: 3px;
                                box-shadow: 3px;
                            """,
                            max_width=800,),
                                highlight_function=lambda x: {'weight':3,'fillColor':'grey'},
                            ).add_to(crimes_per_year)

flm.TileLayer('openstreetmap', overlay=True, name="light mode").add_to(sa_map)
flm.LayerControl(collapsed=False).add_to(sa_map)
sa_map.save('SA_Yearly_Crime_by_Province.html')
sa_map